In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy import read_inventory
client = Client("GEONET")
import pandas as pd

In [2]:
date_text = input("Enter a date (yyyy-mm-dd): ")

Enter a date (yyyy-mm-dd):  2019-06-25


In [3]:
starttime = date_text + " 18:00:00"
cat = client.get_events(starttime=UTCDateTime(starttime), endtime=UTCDateTime(starttime)+86400)

In [50]:
volcano_list = ["Auckland", "Kermadec Islands", "Mayor Island", "Ngauruhoe", "Northland", "Okataina", "Rotorua", "Ruapehu", "Taranaki", "Taupo", "Tongariro", "White Island"]
volc_name = volcano_list
min_lat = [-37.17, -32.9, -37.53, -39.21, -36.03, -38.34, -38.20, -39.5, -39.67, -39.08, -39.21, -38]
max_lat = [-36.57, -24.0, -37.04, -39.08, -34.89, -37.95, -37.98, -39.18,-38.93, -38.5, -39.08,-37.34]
min_lon = [174.45, -174.45, 175.87, 175.56, 173.38, 176.32, 176.12, 175.32, 173.60, 175.56, 175.56, 176.69 ]
max_lon = [175.27, -175.8, 176.64, 175.72, 174.75, 176.81, 176.42, 175.77, 174.49, 176.24, 175.72, 177.40]
volc_df = pd.DataFrame({"volcano_name":volcano_list, "min_lat":min_lat, "max_lat":max_lat, "min_lon":min_lon, "max_lon":max_lon})

In [58]:
for a in range(len(volcano_list)):
    try:
        volcano_list[a] = (volc_name[a],client.get_events(starttime=UTCDateTime(starttime), endtime=UTCDateTime(starttime)+86400, minlatitude = min_lat[a], maxlatitude = max_lat[a], minlongitude = min_lon[a], maxlongitude = max_lon[a], maxdepth = 30))
    except:
        print(volc_name[a] + " No earthquakes found")
filt1 = [x for x in volcano_list if type(x) is tuple] 
anymag = [x for x in filt_volcs if len(x[1]) > 10]
twoplus = []
for a in range(len(filt1)):
    for b in range(len(filt1[a][1])):
        if filt1[a][1][b].preferred_magnitude().mag >= 2:
            twoplus.append(filt1[a][1][b])

Kermadec Islands No earthquakes found
Mayor Island No earthquakes found
Ngauruhoe No earthquakes found
Northland No earthquakes found
Okataina No earthquakes found
Taranaki No earthquakes found
Tongariro No earthquakes found


In [57]:
twoplus

[]

In [29]:
filt1[a][1][0].preferred_magnitude().mag

1.838977797